In [2]:
import py_entitymatching as em
import pandas as pd
import os, sys

In [3]:
A = em.read_csv_metadata('amazon_books_utf8.csv')
B = em.read_csv_metadata('walmart_books_utf8.csv')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [4]:
A['id'] = range(0, len(A))
em.set_key(A, 'id')
B['id'] = range(0, len(B))
em.set_key(B, 'id')

True

In [5]:
sep1 = '–'
sep2 = ':'
sep3 = ')'
sep4 = '('
A['Name'] = A['Name'].apply(lambda x: x.split(sep1, 1)[0])
A['Name'] = A['Name'].apply(lambda x: x.split(sep2, 1)[0])
A['Name'] = A['Name'].apply(lambda x: x.split(sep3, 1)[0])
A['Name'] = A['Name'].apply(lambda x: x.replace(sep4, ''))
A['Publisher'] = A['Publisher'].apply(lambda x: x.split(sep4, 1)[0])
B['Name'] = B['Name'].apply(lambda x: x.split(sep2, 1)[0])

In [30]:
def name_similarity(x, y):
    if len(x['Name'].split()) == 1 and len(y['Name'].split()) == 1:
        if x['Name'].lower().strip() == y['Name'].lower().strip():
            return False
    return True

In [ ]:
# BlackBox Rules
bb = em.BlackBoxBlocker()
bb.set_black_box_function(name_similarity)
C1 = bb.block_tables(A, B, l_output_attrs=['Name', 'Sale Price', 'Category', 'Author', 'ISBN10', 'Pages', 'Publisher', 'Language', 'Dimensions', 'Weight', 'Rating'], r_output_attrs=['Name', 'Sale Price', 'Category', 'Author', 'ISBN10', 'Pages', 'Publisher', 'Language', 'Dimensions', 'Weight', 'Rating'] )

In [263]:
#Overlap Rules
ob = em.OverlapBlocker()
ob.stop_words.append('of')
C2 = ob.block_tables(A, B, 'Author', 'Author',word_level=True, overlap_size=2, l_output_attrs=['Name', 'Sale Price', 'Category', 'Author', 'ISBN10', 'Pages', 'Publisher', 'Language', 'Dimensions', 'Weight', 'Rating'], r_output_attrs=['Name', 'Sale Price', 'Category', 'Author', 'ISBN10', 'Pages', 'Publisher', 'Language', 'Dimensions', 'Weight', 'Rating'] )
#C1 = ob.block_tables(A, B, 'Author', 'Author',word_level=True, overlap_size=2, l_output_attrs=['Name', 'Author', 'ISBN10'], r_output_attrs=['Name','Author', 'ISBN10'] )
C3 = ob.block_candset(C2, 'Name', 'Name',overlap_size=2,word_level=True, rem_stop_words=True)
C4 = em.combine_blocker_outputs_via_union([C1,C3])

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [294]:
#Rule based Rules
block_f = em.get_features_for_blocking(A, B, validate_inferred_attr_types=False)
rb = em.RuleBasedBlocker()
rb.add_rule(['Pages_Pages_anm(ltuple, rtuple) < 0.8'], block_f)
C5 = rb.block_candset(C4, show_progress=True)
rb.add_rule(['Sale_Price_Sale_Price_anm(ltuple, rtuple) < 0.68'], block_f)
C5 = rb.block_candset(C5, show_progress=True)
C5

Column Weight does not seem to qualify as any atomic type. It may contain all NaNs. Please update the values of column Weight
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


,_id,ltable_id,rtable_id,ltable_Name,ltable_Sale Price,ltable_Category,ltable_Author,ltable_ISBN10,ltable_Pages,ltable_Publisher,...,rtable_Sale Price,rtable_Category,rtable_Author,rtable_ISBN10,rtable_Pages,rtable_Publisher,rtable_Language,rtable_Dimensions,rtable_Weight,rtable_Rating
0,0,4,39,The Kremlin Conspiracy Hardcover,18.29,"Books > Mystery, Thriller & Suspense > Thrillers & Suspense",Joel C. Rosenberg,1496406176,NaN,"Tyndale House Publishers, Inc.",...,21.61,Books > Summer Reading Books > Beach Reads,Joel C. Rosenberg,1496406176,480.0,"Tyndale House Publishers, Inc.",English,9.00 x 6.00 x 1.25 Inches,NaN,NaN
1,1,8,963,Origin,16.77,Books > Literature & Fiction > Action & Adventure,Dan Brown,385514239,NaN,Doubleday; 1st Edition edition,...,20.35,Books > Literature & Fiction > Fiction > Action & Adventure,"Brown, Dan",375434542,717.0,Random House Inc,English,9.25 x 6.25 x 1.00 Inches,NaN,NaN
2,2,8,2334,Origin,16.77,Books > Literature & Fiction > Action & Adventure,Dan Brown,385514239,NaN,Doubleday; 1st Edition edition,...,16.50,"Books > Literature & Fiction > Fiction > Mysteries, Thrillers & Suspense > Thrillers & Suspense ...","Brown, Dan",385514239,461.0,Random House Inc,English,9.50 x 6.50 x 1.75 Inches,NaN,4.0000
3,3,9,306,We Were the Lucky Ones,8.97,Books > Literature & Fiction > Genre Fiction,Georgia Hunter,399563091,416.0,Penguin Books; Reprint edition,...,8.97,Books > Literature & Fiction > Fiction > Historical,Georgia Hunter,399563091,416.0,Penguin BooksPenguin Publishing Group,English,8.25 x 5.50 x 1.10 Inches (US),NaN,NaN
4,4,10,373,Fahrenheit 451 Paperback,9.99,Books > Politics & Social Sciences > Politics & Government,Ray Bradbury,1451673310,249.0,Simon & Schuster; Reissue edition,...,9.39,Books > Literature & Fiction > Fiction > Classics > Best seller ranking: > # > 207 > > in > Boo...,Ray Bradbury,1673311,256.0,Simon & Schuster,English,8.44 x 139.70 x 22.86 Inches (US),NaN,4.2857
6,6,11,1301,"Sing, Unburied, Sing",14.79,Books > Literature & Fiction > Genre Fiction,Jesmyn Ward,1501126067,NaN,Scribner; First Edition/First Printing edition,...,21.24,Books > Literature & Fiction > Fiction > Literary,Jesmyn Ward,1501126067,304.0,Scribner,English,8.38 x 139.70 x 1.20 Inches (US),NaN,NaN
8,8,16,1620,Altered Carbon Takeshi Kovacs,12.98,Books > Science Fiction & Fantasy > Science Fiction,Richard K. Morgan,345457684,375.0,Del Rey; 1st edition,...,12.98,Books > Literature & Fiction > Fiction > Science Fiction & Fantasy > Science Fiction > Hard Scie...,"Morgan, Richard K.",5457682,375.0,Random House Inc,English,9.50 x 6.00 x 0.75 Inches,NaN,NaN
16,16,18,1434,A Game of Thrones / A Clash of Kings / A Storm of Swords / A Feast of Crows / A Dance with Drago...,32.49,Books > Literature & Fiction > Action & Adventure,George R. R. Martin,345535529,5216.0,Bantam; Media Tie In edition,...,31.29,Books > Literature & Fiction > Fiction > Action & Adventure,"Martin, George R. R.",449011100,NaN,Random House,English,6.25 x 5.75 x 2.75 Inches,NaN,NaN
23,23,18,2039,A Game of Thrones / A Clash of Kings / A Storm of Swords / A Feast of Crows / A Dance with Drago...,32.49,Books > Literature & Fiction > Action & Adventure,George R. R. Martin,345535529,5216.0,Bantam; Media Tie In edition,...,33.07,Books > Literature & Fiction > Fiction > Science Fiction & Fantasy > Fantasy > Epic,"Martin, George R. R.",307913090,NaN,Random House,English,6.00 x 5.25 x 2.50 Inches,NaN,5.0000
30,30,20,1641,Artemis,16.63,"Books > Mystery, Thriller & Suspense > Thrillers & Suspense",Andy Weir,553448129,NaN,Crown; F First Edition edition,...,16.63,Books > Literature & Fiction > Fiction > Science Fiction & Fantasy > Science Fiction > Action & ...,"Weir, Andy",553448129,305.0,Random House Inc,English,9.50 x 6.25 x 1.25 Inches,NaN,NaN


In [296]:
C5.to_csv('blocked_pairs.csv', index = False, sep = ',')

In [297]:
E = ob.block_candset(C5, 'ISBN10', 'ISBN10',overlap_size=1,word_level=True, rem_stop_words=True)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [298]:
E

,_id,ltable_id,rtable_id,ltable_Name,ltable_Sale Price,ltable_Category,ltable_Author,ltable_ISBN10,ltable_Pages,ltable_Publisher,...,rtable_Sale Price,rtable_Category,rtable_Author,rtable_ISBN10,rtable_Pages,rtable_Publisher,rtable_Language,rtable_Dimensions,rtable_Weight,rtable_Rating
0,0,4,39,The Kremlin Conspiracy Hardcover,18.29,"Books > Mystery, Thriller & Suspense > Thrillers & Suspense",Joel C. Rosenberg,1496406176,NaN,"Tyndale House Publishers, Inc.",...,21.61,Books > Summer Reading Books > Beach Reads,Joel C. Rosenberg,1496406176,480.0,"Tyndale House Publishers, Inc.",English,9.00 x 6.00 x 1.25 Inches,NaN,NaN
2,2,8,2334,Origin,16.77,Books > Literature & Fiction > Action & Adventure,Dan Brown,385514239,NaN,Doubleday; 1st Edition edition,...,16.50,"Books > Literature & Fiction > Fiction > Mysteries, Thrillers & Suspense > Thrillers & Suspense ...","Brown, Dan",385514239,461.0,Random House Inc,English,9.50 x 6.50 x 1.75 Inches,NaN,4.0000
3,3,9,306,We Were the Lucky Ones,8.97,Books > Literature & Fiction > Genre Fiction,Georgia Hunter,399563091,416.0,Penguin Books; Reprint edition,...,8.97,Books > Literature & Fiction > Fiction > Historical,Georgia Hunter,399563091,416.0,Penguin BooksPenguin Publishing Group,English,8.25 x 5.50 x 1.10 Inches (US),NaN,NaN
6,6,11,1301,"Sing, Unburied, Sing",14.79,Books > Literature & Fiction > Genre Fiction,Jesmyn Ward,1501126067,NaN,Scribner; First Edition/First Printing edition,...,21.24,Books > Literature & Fiction > Fiction > Literary,Jesmyn Ward,1501126067,304.0,Scribner,English,8.38 x 139.70 x 1.20 Inches (US),NaN,NaN
30,30,20,1641,Artemis,16.63,"Books > Mystery, Thriller & Suspense > Thrillers & Suspense",Andy Weir,553448129,NaN,Crown; F First Edition edition,...,16.63,Books > Literature & Fiction > Fiction > Science Fiction & Fantasy > Science Fiction > Action & ...,"Weir, Andy",553448129,305.0,Random House Inc,English,9.50 x 6.25 x 1.25 Inches,NaN,NaN
35,35,30,571,The Rooster Bar Hardcover,15.28,"Books > Mystery, Thriller & Suspense > Thrillers & Suspense",John Grisham,385541171,NaN,Doubleday; 1st Edition edition,...,15.28,"Books > Literature & Fiction > Fiction > Mysteries, Thrillers & Suspense > Thrillers & Suspense ...",John Grisham,385541171,368.0,Bantam Dell,English,9.25 x 6.12 x 1.06 Inches,NaN,NaN
37,37,34,112,The Wife Between Us,16.19,Books > Literature & Fiction > Women's Fiction,Greer Hendricks Sarah Pekkanen,1250130921,NaN,St. Martin's Press,...,21.28,Books > Women's History Month Books > Women's History Month Fiction,Sarah Pekkanen Greer Hendricks,1250130921,352.0,St. Martin's Press,English,9.25 x 155.57 x 33.27 Inches (US),NaN,5.0000
44,44,38,2066,The Handmaid's Tale Paperback,9.59,Books > Literature & Fiction > Genre Fiction,Margaret Atwood,038549081X,311.0,Anchor; 1st Anchor Books edition,...,9.59,Books > Walmart Best Sellers > Best seller ranking: > # > 5401 > > in > Books > # > 447 > > in...,"ATWOOD, MARGARET",038549081X,336.0,Random House Inc,English,8.00 x 5.25 x 0.75 Inches,NaN,4.5000
47,47,40,737,Still Me,17.70,Books > Literature & Fiction > Genre Fiction,Jojo Moyes,399562451,NaN,Pamela Dorman Books,...,16.20,Books > Literature & Fiction > Fiction > Contemporary Women,Jojo Moyes,399562451,400.0,Penguin Publishing GroupPamela Dorman Books,English,9.00 x 6.00 x 1.00 Inches (US),NaN,NaN
50,50,42,1343,Red Sparrow,8.30,"Books > Mystery, Thriller & Suspense > Thrillers & Suspense",Jason Matthews,1501168916,576.0,Pocket Books; Media Tie-In edition,...,8.58,Books > Literature & Fiction > Fiction > Media Tie-Ins,Jason Matthews,1501168916,576.0,Pocket Books,English,190.50 x 104.78 x 1.20 Millimeters,NaN,NaN


In [299]:
E
E.to_csv('blocked_pairs_ISBN_match.csv', index = False, sep = ',')

In [23]:
 dbg = em.debug_blocker(C5, A, B)

In [24]:
dbg[['ltable_Name','ltable_ISBN10','ltable_Author','rtable_Name','rtable_ISBN10','rtable_Author']]

,ltable_Name,ltable_ISBN10,ltable_Author,rtable_Name,rtable_ISBN10,rtable_Author
0,The Broken Road,1501111647,Richard Paul Evans,The Mistletoe Promise,1442372680,Richard Paul Evans
1,Persepolis Rising The Expanse,316332836,James S. A. Corey,Cibola Burn,6334685,James S. A. Corey
2,Persepolis Rising The Expanse,316332836,James S. A. Corey,Caliban's War,6129069,James S. A. Corey
3,Persepolis Rising The Expanse,316332836,James S. A. Corey,Abaddon's Gate,6129074,James S. A. Corey
4,Persepolis Rising The Expanse,316332836,James S. A. Corey,Nemesis Games,6334711,James S. A. Corey
5,Persepolis Rising The Expanse,316332836,James S. A. Corey,Babylon's Ashes,316217646,James S. A. Corey
6,Persepolis Rising The Expanse,316332836,James S. A. Corey,Babylon's Ashes,031633474X,James S. A. Corey
7,The Kremlin Conspiracy Hardcover,1496406176,Joel C. Rosenberg,Without Warning,1496423291,Joel C. Rosenberg
8,The Kremlin Conspiracy Hardcover,1496406176,Joel C. Rosenberg,Dead Heat,4311624,Joel C. Rosenberg
9,The Kremlin Conspiracy Hardcover,1496406176,Joel C. Rosenberg,The J. B. Collins Collection,1496427718,Joel C. Rosenberg


In [317]:
S = em.sample_table(C5, 300)
G = em.label_table(S, label_column_name='gold_labels')

Column name (gold_labels) is not present in dataframe
C:\Users\deepa\Anaconda3\lib\site-packages\py_entitymatching\gui\table_gui.py:94: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  table.set_value(idxv[i], cols[j], val)


In [318]:
G

,_id,ltable_id,rtable_id,ltable_Name,ltable_Sale Price,ltable_Category,ltable_Author,ltable_ISBN10,ltable_Pages,ltable_Publisher,...,rtable_Category,rtable_Author,rtable_ISBN10,rtable_Pages,rtable_Publisher,rtable_Language,rtable_Dimensions,rtable_Weight,rtable_Rating,gold_labels
8,8,16,1620,Altered Carbon Takeshi Kovacs,12.98,Books > Science Fiction & Fantasy > Science Fiction,Richard K. Morgan,345457684,375.0,Del Rey; 1st edition,...,Books > Literature & Fiction > Fiction > Science Fiction & Fantasy > Science Fiction > Hard Scie...,"Morgan, Richard K.",5457682,375.0,Random House Inc,English,9.50 x 6.00 x 0.75 Inches,NaN,NaN,1
32,32,25,512,Eleanor Oliphant Is Completely Fine,18.16,Books > Literature & Fiction > Women's Fiction,Gail Honeyman,735220689,NaN,Pamela Dorman Books; F First Edition edition,...,Books > Literature & Fiction > Fiction > Contemporary Women,Gail Honeyman,5220685,336.0,Penguin Publishing GroupPamela Dorman Books,English,NaN,NaN,NaN,1
38,38,35,1118,To Kill a Mockingbird Paperback,8.99,Books > Literature & Fiction > Classics,Harper Lee,60935464,336.0,Harper Perennial Modern Classics,...,Books > Literature & Fiction > Fiction > Literary > Best seller ranking: > # > 45 > > in > Book...,"LEE, HARPER",631078X,384.0,Grand Central Pub,English,6.50 x 4.00 x 1.00 Inches,NaN,4.7000,1
41,41,35,2355,To Kill a Mockingbird Paperback,8.99,Books > Literature & Fiction > Classics,Harper Lee,60935464,336.0,Harper Perennial Modern Classics,...,Books > Literature & Fiction > Fiction > Literary > Best seller ranking: > # > 3061 > > in > Bo...,"LEE, HARPER",1120082,336.0,HarperCollins,English,8.00 x 5.50 x 1.00 Inches,NaN,3.0000,1
44,44,38,2066,The Handmaid's Tale Paperback,9.59,Books > Literature & Fiction > Genre Fiction,Margaret Atwood,038549081X,311.0,Anchor; 1st Anchor Books edition,...,Books > Walmart Best Sellers > Best seller ranking: > # > 5401 > > in > Books > # > 447 > > in...,"ATWOOD, MARGARET",038549081X,336.0,Random House Inc,English,8.00 x 5.25 x 0.75 Inches,NaN,4.5000,1
46,46,39,823,The Great Gatsby Paperback,10.49,Books > Literature & Fiction > Classics,F. Scott Fitzgerald,743273567,180.0,Scribner,...,Books > Literature & Fiction > Fiction > Classics > Best seller ranking: > # > 196 > > in > Boo...,F. Scott Fitzgerald,3273562,192.0,Scribner,English,203.20 x 5.25 x 0.40 Millimeters,NaN,4.8333,1
47,47,40,737,Still Me,17.70,Books > Literature & Fiction > Genre Fiction,Jojo Moyes,399562451,NaN,Pamela Dorman Books,...,Books > Literature & Fiction > Fiction > Contemporary Women,Jojo Moyes,399562451,400.0,Penguin Publishing GroupPamela Dorman Books,English,9.00 x 6.00 x 1.00 Inches (US),NaN,NaN,1
55,55,45,1497,The Hobbit and the Lord of the Rings the Hobbit / the Fellowship of the Ring / the Two Towers / ...,22.99,Books > Literature & Fiction > Genre Fiction,J.R.R. Tolkien,345538374,1728.0,Del Rey,...,Books > Literature & Fiction > Fiction > Science Fiction & Fantasy > Fantasy > Epic > Best selle...,J.R.R. Tolkien,4445786,1504.0,Houghton Mifflin Harcourt,English,5.88 x 107.95 x 3.76 Inches (US),NaN,4.0000,0
58,58,49,2159,Chronicles of Narnia Box Set Paperback,30.64,Books > Children's Books > Science Fiction & Fantasy,C. S. Lewis,61992887,NaN,HarperCollins Narnia; Media Tie In edition,...,Books > Literature & Fiction > Fiction > Science Fiction & Fantasy > Fantasy > Epic,"Lewis, C. S.",7117302,NaN,NaN,NaN,NaN,NaN,NaN,0
60,60,50,1335,Brave New World Paperback,12.99,Books > Science Fiction & Fantasy > Science Fiction,Aldous Huxley,60850523,288.0,Harper Perennial; Reprint edition,...,Books > Literature & Fiction > Fiction > Classics,"Huxley, Aldous",2696125,259.0,HarperCollins,English,9.00 x 6.00 x 1.00 Inches,NaN,NaN,1


In [319]:
G.to_csv('labeled_data.csv', index = False, sep = ',')

In [320]:
train_test = em.split_train_test(G, train_proportion=0.5,random_state=0)
I = train_test['train']
J = train_test['test']

I.to_csv('train.csv')
J.to_csv('test.csv')

In [321]:
F = em.get_features_for_matching(A, B)

Column Weight does not seem to qualify as any atomic type. It may contain all NaNs. Please update the values of column Weight


The table shows the corresponding attributes along with their respective types.
Please confirm that the information  has been correctly inferred.
If you would like to skip this validation process in the future,
please set the flag validate_inferred_attr_types equal to false.


,Left Attribute,Right Attribute,Left Attribute Type,Right Attribute Type,Example Features
0,Name,Name,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
1,Sale Price,Sale Price,numeric,numeric,Exact Match; Absolute Norm
2,Category,Category,medium string (5 words to 10 words),short string (1 word),Not Applicable: Types do not match
3,Author,Author,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
4,ISBN10,ISBN10,short string (1 word),short string (1 word),Levenshtein Distance; Levenshtein Similarity
5,Pages,Pages,numeric,numeric,Exact Match; Absolute Norm
6,Publisher,Publisher,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
7,Language,Language,short string (1 word),short string (1 word),Levenshtein Distance; Levenshtein Similarity
8,Dimensions,Dimensions,medium string (5 words to 10 words),medium string (5 words to 10 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
9,Weight,Weight,short string (1 word to 5 words),un-determined type,Not Applicable: Types do not match


Do you want to proceed? (y/n):y


In [322]:
F

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,Name_Name_jac_qgm_3_qgm_3,Name,Name,qgm_3,qgm_3,jaccard,<function Name_Name_jac_qgm_3_qgm_3 at 0x000002C65251DB70>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
1,Name_Name_cos_dlm_dc0_dlm_dc0,Name,Name,dlm_dc0,dlm_dc0,cosine,<function Name_Name_cos_dlm_dc0_dlm_dc0 at 0x000002C651BFFBF8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
2,Name_Name_jac_dlm_dc0_dlm_dc0,Name,Name,dlm_dc0,dlm_dc0,jaccard,<function Name_Name_jac_dlm_dc0_dlm_dc0 at 0x000002C651BFF9D8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
3,Name_Name_mel,Name,Name,None,None,monge_elkan,<function Name_Name_mel at 0x000002C651BFF620>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
4,Name_Name_lev_dist,Name,Name,None,None,lev_dist,<function Name_Name_lev_dist at 0x000002C651BFF0D0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
5,Name_Name_lev_sim,Name,Name,None,None,lev_sim,<function Name_Name_lev_sim at 0x000002C651BFF598>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
6,Name_Name_nmw,Name,Name,None,None,needleman_wunsch,<function Name_Name_nmw at 0x000002C651C08D08>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
7,Name_Name_sw,Name,Name,None,None,smith_waterman,<function Name_Name_sw at 0x000002C651C08378>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
8,Sale_Price_Sale_Price_exm,Sale Price,Sale Price,None,None,exact_match,<function Sale_Price_Sale_Price_exm at 0x000002C651C08510>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
9,Sale_Price_Sale_Price_anm,Sale Price,Sale Price,None,None,abs_norm,<function Sale_Price_Sale_Price_anm at 0x000002C651C08400>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


In [323]:
F = F.drop(F.index[[44,45,46,47,48,49,50,51,52,53,54,55,56]])
F

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,Name_Name_jac_qgm_3_qgm_3,Name,Name,qgm_3,qgm_3,jaccard,<function Name_Name_jac_qgm_3_qgm_3 at 0x000002C65251DB70>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
1,Name_Name_cos_dlm_dc0_dlm_dc0,Name,Name,dlm_dc0,dlm_dc0,cosine,<function Name_Name_cos_dlm_dc0_dlm_dc0 at 0x000002C651BFFBF8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
2,Name_Name_jac_dlm_dc0_dlm_dc0,Name,Name,dlm_dc0,dlm_dc0,jaccard,<function Name_Name_jac_dlm_dc0_dlm_dc0 at 0x000002C651BFF9D8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
3,Name_Name_mel,Name,Name,None,None,monge_elkan,<function Name_Name_mel at 0x000002C651BFF620>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
4,Name_Name_lev_dist,Name,Name,None,None,lev_dist,<function Name_Name_lev_dist at 0x000002C651BFF0D0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
5,Name_Name_lev_sim,Name,Name,None,None,lev_sim,<function Name_Name_lev_sim at 0x000002C651BFF598>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
6,Name_Name_nmw,Name,Name,None,None,needleman_wunsch,<function Name_Name_nmw at 0x000002C651C08D08>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
7,Name_Name_sw,Name,Name,None,None,smith_waterman,<function Name_Name_sw at 0x000002C651C08378>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
8,Sale_Price_Sale_Price_exm,Sale Price,Sale Price,None,None,exact_match,<function Sale_Price_Sale_Price_exm at 0x000002C651C08510>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
9,Sale_Price_Sale_Price_anm,Sale Price,Sale Price,None,None,abs_norm,<function Sale_Price_Sale_Price_anm at 0x000002C651C08400>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


In [324]:
H = em.extract_feature_vecs(I, 
                            feature_table=F, 
                            attrs_after='gold_labels',
                            show_progress=False) 

# Display first few rows
H.head()

,_id,ltable_id,rtable_id,Name_Name_jac_qgm_3_qgm_3,Name_Name_cos_dlm_dc0_dlm_dc0,Name_Name_jac_dlm_dc0_dlm_dc0,Name_Name_mel,Name_Name_lev_dist,Name_Name_lev_sim,Name_Name_nmw,...,Publisher_Publisher_lev_sim,Publisher_Publisher_nmw,Publisher_Publisher_sw,Language_Language_lev_dist,Language_Language_lev_sim,Language_Language_jar,Language_Language_jwn,Language_Language_exm,Language_Language_jac_qgm_3_qgm_3,gold_labels
1064,1064,1840,94,0.027778,0.000000,0.000000,0.563636,18.0,0.181818,-7.0,...,0.233333,-16.0,6.0,0.0,1.0,1.0,1.0,1.0,1.0,1
111,111,94,199,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,17.0,...,0.187500,4.0,14.0,0.0,1.0,1.0,1.0,1.0,1.0,1
1268,1268,2448,1016,0.354167,0.577350,0.333333,0.877273,27.0,0.386364,-10.0,...,0.258065,-15.0,7.0,0.0,1.0,1.0,1.0,1.0,1.0,1
1208,1208,2190,1977,0.161290,0.377964,0.222222,0.614268,33.0,0.250000,-9.0,...,0.464286,4.0,10.0,0.0,1.0,1.0,1.0,1.0,1.0,0
1336,1336,2788,905,0.441176,0.707107,0.500000,0.900000,15.0,0.500000,0.0,...,0.333333,-5.0,7.0,0.0,1.0,1.0,1.0,1.0,1.0,1


In [325]:
any(pd.notnull(H))

True

In [326]:
H.fillna(value=0, inplace=True)

In [327]:
dt = em.DTMatcher(name='DecisionTree', random_state=0,max_depth=5)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

In [328]:
result_precision = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold_labels'],
        k=5,
        target_attr='gold_labels', metric_to_select_matcher='precision', random_state=0)
result_recall = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold_labels'],
        k=5,
        target_attr='gold_labels', metric_to_select_matcher='recall', random_state=0)
result_f1 = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold_labels'],
        k=5,
        target_attr='gold_labels', metric_to_select_matcher='f1', random_state=0)
result_precision['cv_stats']
result_recall['cv_stats']
result_f1['cv_stats']

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.878323,0.861616,0.868023
1,RF,0.859974,0.873593,0.863892
2,SVM,0.748186,0.990909,0.851043
3,LinReg,0.905514,0.928571,0.915723
4,LogReg,0.906032,0.919481,0.911648
5,NaiveBayes,0.901930,0.855844,0.877045


In [154]:
# Running The matcher

In [329]:
L = em.extract_feature_vecs(J, feature_table=F,
                            attrs_after='gold_labels', show_progress=False)

In [330]:
any(pd.notnull(L))
L.fillna(value=0, inplace=True)

In [331]:
L

,_id,ltable_id,rtable_id,Name_Name_jac_qgm_3_qgm_3,Name_Name_cos_dlm_dc0_dlm_dc0,Name_Name_jac_dlm_dc0_dlm_dc0,Name_Name_mel,Name_Name_lev_dist,Name_Name_lev_sim,Name_Name_nmw,...,Publisher_Publisher_lev_sim,Publisher_Publisher_nmw,Publisher_Publisher_sw,Language_Language_lev_dist,Language_Language_lev_sim,Language_Language_jar,Language_Language_jwn,Language_Language_exm,Language_Language_jac_qgm_3_qgm_3,gold_labels
1058,1058,1817,329,0.448276,0.707107,0.500000,0.904000,12.0,0.520000,1.0,...,0.172414,-19.0,5.0,0.0,1.0,1.0,1.0,1.0,1.0,1
953,953,1596,2194,0.551020,0.745356,0.555556,0.920000,18.0,0.600000,9.0,...,0.142857,-20.0,4.0,0.0,1.0,1.0,1.0,1.0,1.0,0
71,71,59,1143,0.800000,1.000000,1.000000,0.990476,1.0,0.952381,19.0,...,0.459459,10.0,15.0,0.0,1.0,1.0,1.0,1.0,1.0,1
1108,1108,1887,1385,0.511628,0.816497,0.666667,0.911628,19.0,0.558140,5.0,...,0.187500,-14.0,4.0,0.0,1.0,1.0,1.0,1.0,1.0,1
1199,1199,2181,516,0.662162,0.804400,0.647059,0.885965,65.0,0.429825,-16.0,...,0.400000,-2.0,6.0,0.0,1.0,1.0,1.0,1.0,1.0,0
713,713,1105,678,0.418182,0.566947,0.375000,0.894349,24.0,0.510204,2.0,...,0.176471,-16.0,3.0,0.0,1.0,1.0,1.0,1.0,1.0,1
1156,1156,2070,830,0.230769,0.534522,0.285714,0.851429,26.0,0.257143,-17.0,...,0.266667,5.0,5.0,0.0,1.0,1.0,1.0,1.0,1.0,1
1049,1049,1798,634,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,24.0,...,0.228571,-14.0,6.0,0.0,1.0,1.0,1.0,1.0,1.0,1
275,275,314,2031,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,8.0,...,0.120000,-6.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1
560,560,808,1817,0.531250,0.866025,0.750000,0.910345,13.0,0.551724,3.0,...,0.217391,-13.0,5.0,0.0,1.0,1.0,1.0,1.0,1.0,1


In [332]:
J

,_id,ltable_id,rtable_id,ltable_Name,ltable_Sale Price,ltable_Category,ltable_Author,ltable_ISBN10,ltable_Pages,ltable_Publisher,...,rtable_Category,rtable_Author,rtable_ISBN10,rtable_Pages,rtable_Publisher,rtable_Language,rtable_Dimensions,rtable_Weight,rtable_Rating,gold_labels
1058,1058,1817,329,Caliban's War The Expanse,11.55,Books > Science Fiction & Fantasy > Science Fiction,James S. A. Corey,9.78E+12,624.0,Orbit; First Edition edition,...,Books > Literature & Fiction > Fiction > Science Fiction & Fantasy > Science Fiction > Action & ...,James S. A. Corey,6129069,624.0,Orbit,English,9.25 x 6.00 x 1.62 Inches (US),NaN,NaN,1
953,953,1596,2194,Tom Clancy Power and Empire A Jack Ryan Novel,16.76,"Books > Mystery, Thriller & Suspense > Thrillers & Suspense",Marc Cameron,735215898,592.0,G.P. Putnam's Sons; First Edition edition,...,Books > Literature & Fiction > Fiction > War & Military,"Cameron, Marc",525524215,737.0,Random House Inc,English,9.25 x 6.25 x 1.00 Inches,NaN,NaN,0
71,71,59,1143,The Couple Next Door,12.80,"Books > Mystery, Thriller & Suspense > Thrillers & Suspense",Shari Lapena,735221103,336.0,Penguin Books; Reprint edition,...,"Books > Literature & Fiction > Fiction > Mysteries, Thrillers & Suspense > Thrillers & Suspense ...",Shari Lapena,5221102,336.0,Penguin BooksPenguin Publishing Group,English,NaN,NaN,5.000,1
1108,1108,1887,1385,Red Seas Under Red Skies Gentleman Bastards,8.09,Books > Literature & Fiction > Action & Adventure,Scott Lynch,553588958,760.0,Bantam Spectra; Reissue edition,...,Books > Literature & Fiction > Fiction > Science Fiction & Fantasy > Fantasy > Epic > Best selle...,"Lynch, Scott",3588951,760.0,Random House,English,6.75 x 4.10 x 1.20 Inches,NaN,NaN,1
1199,1199,2181,516,"Is It Wrong to Try to Pick Up Girls in a Dungeon?, Vol. 2 - light novel Is It Wrong to Pick Up G...",10.56,Books > Literature & Fiction > World Literature,Fujino Omori,316340146,256.0,Yen On,...,Books > Literature & Fiction > Fiction > Science Fiction & Fantasy > Fantasy > Humorous,Fujino Omori,6318160,240.0,Yen PressYen On,English,NaN,NaN,NaN,0
713,713,1105,678,Interview with the Vampire Mass Market Paperback,7.97,Books > Literature & Fiction > Genre Fiction,Anne Rice,345337662,368.0,Ballantine Books; Reissue edition,...,Books > Literature & Fiction > Fiction > Horror > Best seller ranking: > # > 16031 > > in > Boo...,"Rice, Anne",5337664,346.0,Random House,English,6.75 x 4.25 x 1.25 Inches,NaN,5.000,1
1156,1156,2070,830,"Dead Beat The Dresden Files, Book 7",8.01,"Books > Mystery, Thriller & Suspense > Mystery",Jim Butcher,045146091X,528.0,Roc; English Language edition,...,Books > Literature & Fiction > Fiction > Science Fiction & Fantasy > Fantasy > Paranormal,Jim Butcher,045146091X,528.0,RocPenguin Publishing Group,English,7.56 x 4.25 x 1.13 Inches (US),NaN,NaN,1
1049,1049,1798,634,The Fate of the Tearling,10.87,Books > Science Fiction & Fantasy > Fantasy,Erika Johansen,62290444,496.0,Harper Paperbacks; Reprint edition,...,Books > Literature & Fiction > Fiction > Literary,"Johansen, Erika",229042X,478.0,HarperCollins,English,9.25 x 6.25 x 1.75 Inches,NaN,NaN,1
275,275,314,2031,Uprooted,11.13,Books > Science Fiction & Fantasy > Fantasy,Naomi Novik,804179050,464.0,Del Rey; Reprint edition,...,Books > Literature & Fiction > Fiction > Science Fiction & Fantasy > Fantasy > Epic,"Novik, Naomi",4179055,456.0,Random House Inc,NaN,5.50 x 8.25 x 1.00 Inches,NaN,5.000,1
560,560,808,1817,The Blood Mirror Lightbringer,11.55,Books > Literature & Fiction > Action & Adventure,Brent Weeks,316251321,736.0,Orbit; Reprint edition,...,Books > Literature & Fiction > Fiction > Science Fiction & Fantasy > Fantasy > Epic,Brent Weeks,316251321,736.0,Orbit,English,8.38 x 5.62 x 2.00 Inches (US),NaN,NaN,1


In [335]:
rf.fit(table=H, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold_labels'], target_attr='gold_labels')
# Predict on L 
predictions = rf.predict(table=L,append=True,exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold_labels'],target_attr='predicted_labels')
# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'gold_labels', 'predicted_labels')
em.print_eval_summary(eval_result)

Precision : 89.41% (76/85)
Recall : 81.72% (76/93)
F1 : 85.39%
False positives : 9 (out of 85 positive predictions)
False negatives : 17 (out of 65 negative predictions)


In [336]:
lg.fit(table=H, 
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold_labels'], 
       target_attr='gold_labels')
# Predict on L 
predictions = lg.predict(table=L, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold_labels'], append=True,target_attr='predicted_labels')
# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'gold_labels', 'predicted_labels')
em.print_eval_summary(eval_result)

ValueError: X has 45 features per sample; expecting 44